In [3]:
from pathlib import Path

import click
import numpy as np
import pandas as pd
import rasterra as rt
from rasterio.features import rasterize
from rra_tools import jobmon
import geopandas as gpd

from spatial_temp_cgf import utils
from spatial_temp_cgf import cli_options as clio
from spatial_temp_cgf.data import ClimateMalnutritionData, DEFAULT_ROOT
from spatial_temp_cgf.model_specification import PredictorSpecification, ModelSpecification

from spatial_temp_cgf.inference.run_inference import get_intercept_raster

In [4]:
output_dir = Path(DEFAULT_ROOT)
measure = 'stunting'
model_version = "2024_07_02.02"
cmip6_scenario = 'ssp126'
year = 2025

In [5]:
cm_data = ClimateMalnutritionData(output_dir / measure)
spec = cm_data.load_model_specification(model_version)
print('loading raster template and shapes')
raster_template = cm_data.load_raster_template()
fhs_shapes = cm_data.load_fhs_shapes(most_detailed_only=False)        
print("loading population")
fhs_pop_raster = cm_data.load_population_raster().set_no_data_value(np.nan)
print('loading models')
models = cm_data.load_model_family(model_version)

loading raster template and shapes
loading population
loading models


In [6]:
model_dict = models[0]
model = model_dict['model']

In [14]:
coefs = model.coefs['Estimate']
ranefs = model.ranef

partial_estimates = {}
for predictor in spec.predictors:
    print(predictor.name)
    if predictor.name == 'ldi_pc_pd':
        continue  # deal with after
    elif predictor.name == 'intercept':
        partial_estimates[predictor.name] = get_intercept_raster(
            predictor, coefs, ranefs, fhs_shapes, raster_template,
        )
    else:
        if predictor.random_effect:
            msg = 'Random slopes not implemented'
            raise NotImplementedError(msg)

        if predictor.name == 'elevation':
            v = cm_data.load_elevation()
        else:
            transform = predictor.transform
            variable = (
                transform.from_column
                if hasattr(transform, 'from_column') else predictor.name
            )
            ds = cm_data.load_climate_raster(variable, cmip6_scenario, year)
            v = utils.xarray_to_raster(ds, nodata=np.nan).resample_to(raster_template)

        beta = coefs.loc[predictor.name]
        partial_estimates[predictor.name] = rt.RasterArray(
            beta * np.array(model.var_info[predictor.name]['transformer'](v)),
            transform=v.transform,
            crs=v.crs,
            no_data_value=np.nan,
        )

assert spec.extra_terms == ['any_days_over_30C * ldi_pc_pd']

beta_interaction = (
    coefs.loc['ldi_pc_pd:any_days_over_30C'] / coefs.loc['any_days_over_30C']
)
beta_ldi = (
    beta_interaction * partial_estimates['any_days_over_30C']
    + coefs.loc['ldi_pc_pd']
).to_numpy()
z_partial = sum(partial_estimates.values())

prevalence = 0
for i in range(1, 11):
    print(i)
    ldi = cm_data.load_ldi(year, f"{i / 10.:.1f}")
    
    z_ldi = rt.RasterArray(
        beta_ldi * np.array(model.var_info['ldi_pc_pd']['transformer'](ldi)),
        transform=ldi.transform,
        crs=ldi.crs,
        no_data_value=np.nan,
    )
    prevalence += 0.1 * 1 / (1 + np.exp(-(z_partial + z_ldi)))

return prevalence.astype(np.float32)

intercept
days_over_30C
ldi_pc_pd
any_days_over_30C
1
2
3


KeyboardInterrupt: 

In [13]:
beta_ldi

RasterArray
dimensions    : 36000, 18000 (x, y)
resolution    : 0.01, -0.01 (x, y)
extent        : -180.0, 180.0, -90.0, 90.0 (xmin, xmax, ymin, ymax)
crs           : EPSG:4326
no_data_value : nan
size          : 4943.85 MB
dtype         : float64